<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(5pm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml
!pip install parse

    100% |████████████████████████████████| 5.8MB 7.2MB/s 
  Running setup.py bdist_wheel for parse ... - done
  Stored in directory: /root/.cache/pip/wheels/ef/db/c6/18568a2cc574848f3996ac4552241fbec046b7be29feb2077d
Successfully built parse


In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 28 #limit to check whether they are relative or not
NOISE = 9

In [3]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 연신내 맥도날드
2 -> 이창섭
3 -> 공범
4 -> 스즈키컵
5 -> 화웨이
6 -> 2018 스즈키컵
7 -> 굿바이싱글
8 -> 김지명
9 -> 홍수현
10 -> 영화 공범
11 -> 권다현
12 -> 김미화
13 -> 영리병원
14 -> 선덕고 김지명
15 -> 삼성전자 임원인사
16 -> 견미리
17 -> 삼성 임원인사
18 -> 베트남 필리핀
19 -> 나비
20 -> 손흥민 골


In [4]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
      
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

2018 스즈키컵  and  스즈키컵
영화 공범  and  공범
선덕고 김지명  and  김지명
베트남 필리핀  and  스즈키컵


In [5]:

print('\n',label,'\n')

for i in range(20):
  print(matrix[i])

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])
 
print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 4, 7, 8, 9, 3, 11, 12, 13, 8, 15, 16, 17, 4, 19, 20] 

[0, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 0, 9, 0, 0, 0, 9, 0, 6, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0]
[4, 0, 9, 0, 0, 6, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 100, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 6, 0, 0]
[4, 0, 11, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 2, 11, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 155, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 95, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 9, 9, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0]
[4, 0

In [6]:

#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  연신내 맥도날드
2 .  이창섭
3 .  스즈키컵 | 2018 스즈키컵 | 베트남 필리핀
4 .  공범 | 영화 공범
5 .  화웨이
6 .  김지명 | 선덕고 김지명
7 .  굿바이싱글
8 .  홍수현
9 .  권다현
10 .  김미화
11 .  영리병원
12 .  삼성전자 임원인사
13 .  견미리
14 .  삼성 임원인사
15 .  나비
16 .  손흥민 골
